In [0]:
import os
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from matplotlib import image as mpimg
from PIL import Image
from keras.layers import *
from keras.models import *
from keras.optimizers import Adam


In [0]:
# Kreiranje numpy nizova od dobijenih slika

gcolab_valdo_putanja = os.path.join ('Hey-Waldo', '64', 'gen_waldo', '')
gcolab_ne_valdo_putanja = os.path.join ('Hey-Waldo', '64', 'notwaldo', '')

valdo = np.array([np.array(Image.open(gcolab_valdo_putanja + ime_slike)) for ime_slike in os.listdir(gcolab_valdo_putanja)])
ne_valdo = np.array([np.array(Image.open(gcolab_ne_valdo_putanja + ime_slike)) for ime_slike in os.listdir(gcolab_ne_valdo_putanja)])


In [0]:
# Deljenje na skupove

y1 = [1 for i in range(len(valdo))]
y2 = [0 for i in range(len(ne_valdo))]
y = np.array(y1 + y2)

X = np.concatenate ((valdo, ne_valdo))

x_trening_valid, x_test, y_trening_valid, y_test = ms.train_test_split(X, y, train_size = 0.67, test_size=0.33, stratify = y, random_state = 3)
x_trening, x_valid, y_trening, y_valid = ms.train_test_split(x_trening_valid, y_trening_valid, train_size = 0.7, test_size = 0.3, stratify = y_trening_valid, random_state = 3)


In [0]:
# Centriranje

x_trening = x_trening / 255
x_trening_valid = x_trening_valid / 255
x_test = x_test / 255


In [0]:
# Kreiranje modela

knn = Sequential()

knn.add(Conv2D(filters = 32, 
               kernel_size = (2, 2), 
               strides = (1,1),
               padding = 'same',
               input_shape = (64, 64, 3),
               data_format = 'channels_last'))

knn.add(Activation('relu'))

knn.add(MaxPooling2D(pool_size = (2,2), strides = 2))

knn.add(Dropout(0.4))

knn.add(Conv2D(filters = 64,
               kernel_size = (2,2),
               strides = (1,1),
               padding = 'valid'))

knn.add(Activation('relu'))

knn.add(MaxPooling2D(pool_size = (2,2), strides = 2))

knn.add(Flatten())

knn.add(Dense(64))

knn.add(Activation('relu'))

knn.add(Dropout(0.4))

knn.add(Dense(1))

knn.add(Activation('sigmoid'))


In [0]:
# Treniranje modela

knn.compile (optimizer = Adam(lr=0.00075), loss='binary_crossentropy', metrics=['accuracy'])

knn.fit (x = x_trening, y = y_trening, batch_size=64, epochs=50, validation_data = (x_valid, y_valid))



In [0]:
# Evaluacija modela

greska, tacnost = knn.evaluate(x_test, y_test)

print(f'Greska: {greska}, tacnost: {tacnost}')
